TODO for Pania demo
- [ok] open scene and manipulate it
- [ok] translate slices
- [ok] rotate camera
- [ok] take snapshot 
- [ok] change isovalue
- [todo] change slice palette
- [todo] change color iso

In [1]:
import os,sys,pickle,random,threading,time
import platform,subprocess,glob,datetime
import numpy as np
from PyQt5 import QtCore
from PyQt5 import QtWidgets
import sip

# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
%gui qt
%matplotlib qt
print([it for it in sys.modules.keys() if "pyqt" in it.lower()])

# in case you are in debug mode
sys.path.append(r"C:\projects\OpenVisus\build\RelWithDebInfo")

# change working directory as needed
os.chdir(r"C:\projects\OpenVisus")
print("current working directory",os.getcwd())

from OpenVisus                        import *
from OpenVisus.gui                    import *
from OpenVisus.image_utils            import *

# use this function to create a Viewer, solves the problem of window not raising
viewer=None
def CreateViewer():
    global viewer
    
    if viewer is not None:
        del viewer
    
    viewer=PyViewer()
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    viewer_py.setVisible(True)
    viewer_py.show()
    viewer_py.setFocus()
    viewer_py.showMaximized()
    viewer_py.activateWindow()
    print(time.time(),"Viewer created")
    return viewer

['PyQt5', 'PyQt5.sip', 'PyQt5.QtCore', 'PyQt5.QtGui', 'PyQt5.QtWidgets']
current working directory C:\projects\OpenVisus
Starting OpenVisus C:\projects\OpenVisus\build\RelWithDebInfo\OpenVisus\__init__.py 3.9.11 (tags/v3.9.11:2de452f, Mar 16 2022, 14:33:45) [MSC v.1929 64 bit (AMD64)] sys.version_info(major=3, minor=9, micro=11, releaselevel='final', serial=0) ...
QT_PLUGIN_PATH C:\Python39\lib\site-packages\PyQt5\Qt5/plugins


In [2]:
# do not change this cell for very good reasons (otherwise Qt5 loop won't work)
# this is needed to run the Qt5 loop in the background 
# see http://localhost:8888/notebooks/VisibleMale.ipynb#
import time
time.sleep(2)

Define some utilities to perform viewer animations

In [6]:
# //////////////////////////////////////////////////////////////
"""
RunActions is needed to schedule actions and execute them as soon as the viewer is idle
Example:

    run_actions.addAction(fn,arg=0)

will run later.
If you want to execute an immediate action:

    fn(arg)

"""
class RunActions:
    
    def __init__(self):
        self.v=[]
        self.cursor=0
        self.timer=QtCore.QTimer()
        self.timer.timeout.connect(self.onTimer)
  
    def addAction(self, fn,**kwargs):
        self.v.append(lambda : fn(**kwargs))
        
    def addSleep(self,msec):
        self.v.append(msec)
        
    def onTimer(self):
        
        # if the last action was an action and the viewer is still running, I need to wait
        if self.cursor>0 and callable(self.v[self.cursor-1]) and viewer.isRunning():
            return
        
        if self.cursor>= len(self.v):
            return
        
        cur=self.v[self.cursor]
        self.cursor+=1
        
        # is a delay?
        if isinstance(cur,int):
            print("*** Sleep",cur)
            self.timer.start(cur) 
            
        # is a real action, run it and later wait for completition
        elif callable(cur):
            cur()
            self.timer.start(20) 
            
        # there must be a problem
        else:
            raise Exception("internal error")
   
    def start(self): 
        print("RunAction started")
        self.timer.start(1) 
        
    def stop(self):
        print("RunAction stopped")
        self.timer.stop()
        
        
def RotateScene(axis=(0,0,1),angle=10):
    print("*** RotateScene",axis,angle)
    glcamera=viewer.getGLCamera()
    viewer.getGLCamera().setRotation(glcamera.getRotation() * Quaternion(Point3d(axis),math.radians(angle)))
    viewer.refreshAll()
    viewer.postRedisplay()
    

    
def TakeSnapshot(filename="temp.png",):
    print("*** TakeSnapshotAction",filename)  
    # viewer.takeSnapshot(False,filename) BROKEN
    viewer_py=sip.wrapinstance(FromCppQtWidget(viewer.c_ptr()), QMainWindow)
    screenshot = QtWidgets.QApplication.primaryScreen().grabWindow(viewer_py.winId() )
    screenshot.save(filename)
    
def OpenScene(filename=""):
    print("*** OpenScene",filename)
    viewer.open(filename)
    
def DropSelection():
    print("*** DropSelection")
    viewer.dropSelection()
    
def HideDatasetBounds(uuid="dataset"):
    print("*** HideDatasetBounds",uuid)
    # **** make sure that the dataset node has UUID `dataset` (you can save an xml and inspect it) ***
    dataset_node=DatasetNode.castFrom(viewer.findNodeByUUID(uuid))
    if dataset_node:
        dataset_node.setShowBounds(False)
    else:
        print("Failed to find a node with uuid",uuid)
      
def TranslateSlice(slice=None,axis=0,offset=0):
    viewer.setSlicePosition(slice, axis, offset)

# run_actions.addAction(TakeSnapshot,filename="example.png")
# TakeSnapshot(filename="example.png")
    
print(time.time(),"Utilities defined")

1663708106.5612545 Utilities defined


In [4]:
# important to create the viewer in a different cell otherwise it won't get the focus
viewer=CreateViewer()

1663708041.9437466 Viewer created


In [5]:
run_actions=RunActions()
run_actions.start()
run_actions.addAction(OpenScene,filename=r"C:\data\visus-datasets\Pania_2021Q3_in_situ_data\idx\fly_scan_id_112524.h5\segmentations\modvisus\visus.idx")

# default for addXXX functions
parent=None
fieldname=""
access_id=0

print(time.time(),"OpenScene")

RunAction started
1663708046.3860872 OpenScene
*** OpenScene C:\data\visus-datasets\Pania_2021Q3_in_situ_data\idx\fly_scan_id_112524.h5\segmentations\modvisus\visus.idx
2022-09-20 15:07:26.507512 PyScriptingNode Got in input (34, 32, 63) uint8 origin Volume 
2022-09-20 15:07:26.507512 PyScriptingNode Output is  (34, 32, 63) uint8 msec 0 


In [6]:
# uncomment as needed
# run_actions.addAction(DropSelection)
# run_actions.addAction(HideDatasetBounds,uuid="dataset")

In [7]:
# remove default renderer
viewer.removeNode("volume")

In [8]:
# add volume render
volume=viewer.addVolume("volume", parent, fieldname, access_id)
volume.setName("volume")

2022-09-20 15:02:07.016044 PyScriptingNode Got in input (34, 32, 63) uint8 origin volume 
2022-09-20 15:02:07.017798 PyScriptingNode Output is  (34, 32, 63) uint8 msec 1 


In [7]:
# X slice
xslice=viewer.addSlice("xslice", None, fieldname, access_id, 0)
xslice.setName("xslice")
viewer.setSlicePosition(xslice, 0, 200)

2022-09-20 15:08:37.762501 PyScriptingNode Got in input (1, 36, 37) uint8 origin xslice 
2022-09-20 15:08:37.764578 PyScriptingNode Output is  (1, 36, 37) uint8 msec 2 


In [10]:
# y slice
yslice=viewer.addSlice("yslice", None, fieldname, access_id, 1)
yslice.setName("yslice")
viewer.setSlicePosition(yslice, 1, 200)

2022-09-20 15:02:15.641169 PyScriptingNode Got in input (1, 37, 36) uint8 origin yslice 
2022-09-20 15:02:15.642125 PyScriptingNode Output is  (1, 37, 36) uint8 msec 1 


In [11]:
# z slice
zslice=viewer.addSlice("zslice", None, fieldname, access_id, 2)
zslice.setName("zslice")
viewer.setSlicePosition(zslice, 2, 200)

2022-09-20 15:02:20.073518 PyScriptingNode Got in input (1, 36, 36) uint8 origin zslice 
2022-09-20 15:02:20.074729 PyScriptingNode Output is  (1, 36, 36) uint8 msec 2 


In [13]:
#viewer.removeNode("xslice")
#viewer.removeNode("yslice")
#viewer.removeNode("zslice")
#viewer.removeNode("volume")

In [14]:
# add isosurface
isovalue=100
iso=viewer.addIsoContour("iso", parent, fieldname, access_id, str(isovalue))
iso.setName("iso")

# if you later want to remove it
# viewer.removeNode("iso")

2022-09-20 15:03:17.632446 PyScriptingNode Got in input (68, 63, 125) uint8 origin iso 
2022-09-20 15:03:17.634858 PyScriptingNode Output is  (68, 63, 125) uint8 msec 2 


In [15]:
# change isovalue
dataflow=viewer.getDataflow()
v=[IsoContourNode.castFrom(it) for it in dataflow.getNodesAsVector() if IsoContourNode.castFrom(it)]
v[0].setIsoValue(200)
print("New isovalue",v[0].getIsoValue())

New isovalue 200.0


In [8]:
# example of rotation of the scene of 360 degrees
for I in range(10):
    run_actions.addAction(RotateScene,axis=(0,0,1),angle=10)
    run_actions.addAction(TranslateSlice,slice=xslice,axis=0,offset=I*100)
    run_actions.addAction(TakeSnapshot,filename=os.path.join(os.getcwd(),f"snapshot.{I:03d}.png"))
    # important to sleep, otherwise take snapshot will not work (I know it makes no sense)
    run_actions.addSleep(50)

*** RotateScene (0, 0, 1) 10
2022-09-20 15:09:46.322612 PyScriptingNode Got in input (1, 1, 33) uint8 origin xslice 
2022-09-20 15:09:46.323009 PyScriptingNode Output is  (1, 1, 33) uint8 msec 1 
2022-09-20 15:09:46.323282 PyScriptingNode Got in input (34, 32, 63) uint8 origin Volume 
2022-09-20 15:09:46.323282 PyScriptingNode Output is  (34, 32, 63) uint8 msec 0 
*** TakeSnapshotAction C:\projects\OpenVisus\snapshot.000.png
*** Sleep 50
*** RotateScene (0, 0, 1) 10
2022-09-20 15:09:46.636289 PyScriptingNode Got in input (34, 32, 63) uint8 origin Volume 
2022-09-20 15:09:46.636289 PyScriptingNode Output is  (34, 32, 63) uint8 msec 0 
2022-09-20 15:09:46.636289 PyScriptingNode Got in input (1, 40, 36) uint8 origin xslice 
2022-09-20 15:09:46.636872 PyScriptingNode Output is  (1, 40, 36) uint8 msec 1 
*** TakeSnapshotAction C:\projects\OpenVisus\snapshot.001.png
*** Sleep 50
*** RotateScene (0, 0, 1) 10
2022-09-20 15:09:46.889934 PyScriptingNode Got in input (1, 41, 36) uint8 origin xsli

In [ ]:
# you may want to create several scenes and play
if False:
    snapshots=sorted(glob.glob(r"D:\visus_demo\viewer_control_from_jupyter\honeycomb1\visusviewer.snapshot*xml"))

    run_actions.stop()
    
    # limit for debugging...
    snapshots=snapshots[:2] 

    for I,snapshot in enumerate(snapshots):
        print("\t",I,snapshot)
        run_actions.addAction(OpenScene,filename=snapshot)
        run_actions.addAction(DropSelection)
        run_actions.addAction(HideDatasetBounds,uuid="dataset")
        run_actions.addAction(TakeSnapshot,filename=os.path.splitext(snapshot)[0] + ".png")

        # you can add some sleep between actions but it's not stricly necessary:
        # RunActions waits for the viewer to become idle
        run_actions.addSleep(10) 

    run_actions.start()